<a href="https://colab.research.google.com/github/fran1215/MLAI-Project-Team3/blob/main/light_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data


In [24]:
cd/content/drive/MyDrive

/content/drive/MyDrive


In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, plot_roc_curve
import warnings
from sklearn.model_selection import KFold

warnings.filterwarnings("ignore")

In [45]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

train=train.drop(['Unnamed: 0', 'id'],axis=1).reset_index(drop=True)
test=test.drop(['Unnamed: 0', 'id'],axis=1).reset_index(drop=True)
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [46]:
def preprocess(data):
    
    data=data.dropna().reset_index(drop=True)
    label_list=['Gender','Customer Type','Type of Travel','Class','satisfaction']
    
    for i in label_list:
        y1=data[[i]].values.tolist()
        encoder=LabelEncoder()
        encoder.fit(y1)
        label1=encoder.transform(y1)
        data[i]=label1
        
    return data

#Modeling

#light GBM

In [47]:
train_clean=preprocess(train)
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103594 entries, 0 to 103593
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103594 non-null  int64  
 1   Customer Type                      103594 non-null  int64  
 2   Age                                103594 non-null  int64  
 3   Type of Travel                     103594 non-null  int64  
 4   Class                              103594 non-null  int64  
 5   Flight Distance                    103594 non-null  int64  
 6   Inflight wifi service              103594 non-null  int64  
 7   Departure/Arrival time convenient  103594 non-null  int64  
 8   Ease of Online booking             103594 non-null  int64  
 9   Gate location                      103594 non-null  int64  
 10  Food and drink                     103594 non-null  int64  
 11  Online boarding                    1035

In [48]:
X = train_clean.drop('satisfaction',axis=1)
Y = train_clean['satisfaction']

In [49]:
kfold=KFold(n_splits=5)
cv_score=[]

x=X
y=Y
model= LGBMClassifier(iterations=100)
n_iter=0
for train_index, test_index in kfold.split(x):
    x_train, x_test=x.iloc[train_index],x.iloc[test_index]
    y_train, y_test=y[train_index],y[test_index]
    
    model.fit(x_train,y_train)
    y_pred_train=model.predict(x_train)
    y_pred_test=model.predict(x_test)
    
    n_iter=n_iter+1
    
    acc_train=accuracy_score(y_train,y_pred_train)
    acc_test=accuracy_score(y_test,y_pred_test)
    cv_score.append(acc_test)
    
    print("Modeling ",n_iter,"st")
    print('train set accuracy_score: ',acc_train)
    print('test set accuracy_score: ',acc_test)
    print("-----------------------------------------")
    
print('Mean test set accuracy_score: ',np.mean(cv_score))

Modeling  1 st
train set accuracy_score:  0.9671915535444947
test set accuracy_score:  0.9626429847000337
-----------------------------------------
Modeling  2 st
train set accuracy_score:  0.9665641025641025
test set accuracy_score:  0.9634634876200588
-----------------------------------------
Modeling  3 st
train set accuracy_score:  0.9670346907993966
test set accuracy_score:  0.9633669578647618
-----------------------------------------
Modeling  4 st
train set accuracy_score:  0.9665641025641025
test set accuracy_score:  0.9642357256624354
-----------------------------------------
Modeling  5 st
train set accuracy_score:  0.96663690332545
test set accuracy_score:  0.9657302828458345
-----------------------------------------
Mean test set accuracy_score:  0.9638878877386249


In [57]:
from sklearn.model_selection import GridSearchCV

model=LGBMClassifier()
param={'min_data_in_leaf':[20,30,50],'max_depth':[-1,8,15],'iterations':[150,200],'learning_rate':[0.09,0.5,0.8],'num_leaves':[31,20,10]}

grid_model=GridSearchCV(model,param_grid=param, cv=5,scoring='f1')
grid_model.fit(X,Y)
result1=pd.DataFrame(grid_model.cv_results_)

In [58]:
grid_model.best_params_

{'iterations': 150,
 'learning_rate': 0.09,
 'max_depth': -1,
 'min_data_in_leaf': 30,
 'num_leaves': 31}

In [59]:
test_clean = preprocess(test)

X_test = test_clean.drop('satisfaction',axis=1)
Y_test = test_clean['satisfaction']

In [60]:
model = LGBMClassifier(**grid_model.best_params_)
model.fit(X, Y)
pred = model.predict(X_test)

In [61]:
accuracy_score(Y_test,pred)

0.96392847487738